# 7.4 Convolution / Pooling レイヤの実装
## 7.4.1 4次元配列

In [2]:
import numpy as np

x = np.random.rand(10, 1, 28, 28) # ランダムにデータを生成
x.shape

(10, 1, 28, 28)

In [3]:
x[0].shape
x[1].shape

(1, 28, 28)

In [4]:
x[0,0]

array([[  4.83420947e-01,   9.64735568e-01,   7.43932088e-01,
          5.51932498e-01,   5.98083620e-01,   1.26990822e-01,
          8.18057614e-01,   6.51894016e-02,   8.11707076e-01,
          6.41799302e-01,   5.94612697e-02,   7.67986021e-01,
          3.74198527e-01,   3.29999780e-01,   8.78573413e-02,
          1.43665735e-01,   4.95217867e-01,   1.33004775e-02,
          8.25000522e-01,   1.74543130e-01,   5.80771292e-01,
          2.58465955e-01,   7.78191409e-01,   7.65888799e-01,
          7.02555675e-01,   9.63747933e-01,   5.92202034e-01,
          8.71917696e-01],
       [  6.11037230e-02,   6.42742872e-01,   9.18888774e-01,
          2.37297194e-01,   2.91952970e-01,   4.76540200e-01,
          7.02966922e-01,   9.90919616e-01,   8.80708311e-02,
          5.43739622e-01,   6.17031951e-01,   8.91284163e-01,
          6.96364500e-01,   8.60650289e-01,   9.22781860e-01,
          1.89635475e-01,   4.21684397e-01,   9.22945033e-01,
          3.95842442e-02,   9.39517052e-01,

## 7.4.2 im2col による展開

In [8]:
import sys, os
sys.path.append(os.pardir)
from common.util import im2col

x1 = np.random.rand(1, 3, 7, 7)
col1 = im2col(x1, 5, 5, stride=1, pad=0)
print(col1.shape)

(9, 75)


In [9]:
x2 = np.random.rand(10, 3, 7, 7) # 10個のデータ
col2 = im2col(x2, 5, 5, stride=1, pad=0)
print(col2.shape)

(90, 75)


In [10]:
class Covolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride = stride
        self.pad = pad
    
    def forward(self, x):
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1 + (H + 2*self.pad - FH) / self.stride)
        out_w = int(1 + (W + 2*self.pad - FW) / self.stride)
        
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T # フィルターの展開
        out = np.dot(col, col_W) + self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0, 3, 1, 2)
        
        return out

## 7.4.4 Pooling レイヤの実装

In [12]:
class Pooling:
        def __init__(self, pool_h, pool_w, stride=1, pad=0):
            self.pool_h = pool_h
            self.pool_w = pool_w
            self.stride = stride
            self.pad = pad

        def forward(self, x):
            N, C, H, W = x.shape
            out_h = int(1 + (H - self.pool_h) / self.stride)
            out_w = int(1 + (W - self.pool_w) / self.stride)
            
            # 1.展開
            col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
            col = col.reshape(-1, self.pool_h*self.pool_w)
            
            # 2.最大値
            out = np.max(col, axis=1)
            
            # 3.整形
            out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1, 2)
            
            return out

# 7.5 CNNの実装

In [16]:
from collections import OrderedDict
from common.layers import *

class SimpleConvNet:
        def __init__(self, input_dim=(1, 28, 28),
                    conv_param={'filter_num':30, 'filter_size':5, 'pad':0, 'stride':1},
                    hidden_size=100, output_size=10, weight_init_std=0.01):
            filter_num = conv_param['filter_num']
            filter_size = conv_param['filter_size']
            filter_pad = conv_param['pad']
            filter_stride = conv_param['stride']
            input_size = inpu[1]
            conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride + 1
            pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
            
            # 重みの初期化
            self.params = {}
            self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
            self.params['b1'] = np.zeros(filter_num)
            self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
            self.params['b2'] = np.zeros(hidden_size)
            self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
            self.params['b3'] = np.zeros(output_size)
            
            # レイヤの生成
            self.layers = OrderedDict()
            self.layers['Conv1'] = Convolution(self.params['W1'],
                                              self.params['b1'],
                                              conv_param['stride'],
                                              conv_param['pad'])
            self.layers['Relu1'] = Relu()
            self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
            self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
            self.layers['Relu2'] = Relu()
            self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
            
            self.last_layer = SoftmaxWithLoss()
        
        def predict(self, x):
            for layer in self.layers.values():
                x = layer.forward(x)
            return x
        
        def loss(self, x, t):
            y = self.predict(x)
            return self.lastLayer.forward(y, t)
        
        def gradient(self, x, t):
            # forward
            self.loss(x, t)
            
            # backward
            dout = 1
            dout = self.last_layer.backward(dout)
            
            layers = list(self.layers.values())
            layers.reverse()
            for layer in layers:
                dout = layer.backward(dout)
                
            # 設定
            grads = {}
            grads['W1'] = self.layers['Conv1'].dW
            grads['b1'] = self.layers['Conv1'].db
            grads['W2'] = self.layers['Affine1'].dW
            grads['b2'] = self.layers['Affine1'].db
            grads['W3'] = self.layers['Affine2'].dW
            grads['b3'] = self.layers['Affine2'].db
            
            return grads